### Deep Neural Decision Trees: tensorflow code

* DNDT Paper: https://arxiv.org/pdf/1806.06988.pdf
* code: https://github.com/wOOL/DNDT/blob/master/tensorflow/neural_network_decision_tree.py#L10
* Ref Notebook: https://github.com/wOOL/DNDT/blob/master/tensorflow/demo.ipynb


* **Key parameters defined for `nn_decision_tree(. . .)`:**
    * `x`: Placeholder for 2 input feature values (Petal length, Petal width) in x
    * `cut_points_list`: Variable holding randomly inititialized n `cut point values`, a trainable param.
    * `leaf_score`: Variable holding trainable weights/values mapping last layer bins to output target class layer

In [2]:
import numpy as np
import tensorflow as tf
import iris

In [3]:
tf.__version__

'2.0.0'

In [6]:
x = iris.feature[:, 2:4]  # use "Petal length" and "Petal width" only
y = iris.label
d = x.shape[1]

In [7]:
num_cut = [1, 1]  # "Petal length" and "Petal width"
num_leaf = np.prod(np.array(num_cut) + 1)# 4
num_class = 3

In [11]:
cut_points_list = [tf.Variable(tf.random.uniform([i])) for i in num_cut]
leaf_score = tf.Variable(tf.random.uniform([num_leaf, num_class]))

In [14]:
cut_points_list#with TF 2.0

[<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.38126063], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.08172297], dtype=float32)>]

In [15]:
leaf_score#from TF 2.0 eager execution 

<tf.Variable 'Variable:0' shape=(4, 3) dtype=float32, numpy=
array([[0.6178036 , 0.23387122, 0.1611011 ],
       [0.3350451 , 0.36053157, 0.40758777],
       [0.83489025, 0.19071114, 0.0848999 ],
       [0.04420972, 0.41416192, 0.6197046 ]], dtype=float32)>

* **Starting with `nn_decision_tree( . . .)`**

* **In line 1:**

`leaf = reduce(tf_kron_prod, map(lambda z: tf_bin(x[:, z[0]:z[0] + 1], z[1], temperature), enumerate(cut_points_list)))`.

* `lambda` function passes `cut_points_list` values  one by one, alongside a corresponding column of feature vector x, say `x[:,0]` representing `Petal length` into `tf_bin(. . .)` functon; i.e., first passing  `cut_points_list[0]`with `x[:,0:1]`; And then `cut_points_list[1]`and `x[:,1:2]`.
                  
* The obtained sequence of softmax outputs of two `input features` & `two cut_points` through `tf_bin( )` is then passed into `tf_kron_prod(. . .)`
* `result = map(lambda z: tf_bin(x[:, z[0]:z[0] + 1], z[1], 0.1), enumerate(cut_points_list))`

####  #1.

In [16]:
#1.
def nn_decision_tree(x, cut_points_list, leaf_score, temperature=0.1):
    # cut_points_list contains the cut_points for each dimension of feature
    leaf = reduce(tf_kron_prod,
                  map(lambda z: tf_bin(x[:, z[0]:z[0] + 1], z[1], temperature), enumerate(cut_points_list)))
    return tf.matmul(leaf, leaf_score)


#y_pred = nn_decision_tree(x_ph, cut_points_list, leaf_score, temperature=0.1)


**So `z` is arg passed into lambda from iterable `enumerate(cut_points_list)`**.
* And `z[0]` & `z[1]` correspond to first & second `cut_points` respectively.
* And `z[0][0]` = index & `z[0][1]` = underlying cut_point tensors

In [20]:
z= enumerate(cut_points_list)
cut_list= list(z)
print('cut_point_list:\n',cut_list, '\n\ncut_point 1:\n', cut_list[0][1], '\ncut_point 2:\n', cut_list[1][1])

cut_point_list:
 [(0, <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.38126063], dtype=float32)>), (1, <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.08172297], dtype=float32)>)] 

cut_point 1:
 <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.38126063], dtype=float32)> 
cut_point 2:
 <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.08172297], dtype=float32)>


In [22]:
#x[:,0:1]#correspondes to x[:, z[0]:z[0] + 1], z[1] above
#x[:,1:2]#correspondes to x[:, z[1]:z[1] + 1], z[1] above

**Now the above params are passed into following `tf_bin( )` function, one at a time resulting in softmax outputs**
* the `map(lambda z: tf_bin(x[:, z[0]:z[0] + 1], z[1], temperature), enumerate(cut_points_list))` passes -- `feature 1(Petal length)` i.e., `x[:,0:1]` as `x[:, z[0]:z[0] + 1]` and corresponding `cut_list[0][1]` as `z[1]` into **`tf_bin( )`** first then `2nd feature (Petal width)` & 2nd cut_point.

#####  #2.

In [26]:
#2.
def tf_bin(x, cut_points, temperature=0.1):
    # x is a N-by-1 matrix (column vector), corresponding to 1 feature at a time
    # cut_points is a D-dim vector (D is the number of cut-points)
    # this function produces a N-by-(D+1) matrix, each row has only one element being one and the rest are all zeros
    D = cut_points.get_shape().as_list()[0]
    W = tf.reshape(tf.linspace(1.0, D + 1.0, D + 1), [1, -1])
    cut_points = tf.sort(cut_points)  # make sure cut_points is monotonically increasing
    b = tf.cumsum(tf.concat([tf.constant(0.0, shape=[1]), -cut_points], 0))
    h = tf.matmul(x, W) + b
    res = tf.nn.softmax(h / temperature)
    return res

#tf_bin(x[:, z[0]:z[0] + 1], z[1], 0.1)

* **`tf_bin( )` results for `feature 1(Petal length)` i.e., `x_ph[:,0:1]` and `1st cut_point` or `z[0]`, i.e.,`cut_list[0][1]` from above.**
    * Since the `tf_bin( )` is invoked two times, one for each `x` feature 1 & 2, also for each item from cut_point_list. Therefore computing softmax at each one's end.

In [24]:
cut_list[0][1].get_shape().as_list()#value of D

[1]

In [25]:
D=1
W= tf.reshape(tf.linspace(1.0, D + 1.0, D + 1), [1, -1])
W

<tf.Tensor: id=63, shape=(1, 2), dtype=float32, numpy=array([[1., 2.]], dtype=float32)>

In [27]:
cut_pts= tf.sort(cut_list[0][1])
cut_pts

<tf.Tensor: id=75, shape=(1,), dtype=float32, numpy=array([0.38126063], dtype=float32)>

* Following lines up sorted `cut_point` values as `[0,-b1,-b2-, . ., -bn]` as mentioned in paper.

In [29]:
tf.concat([tf.constant(0.0, shape=[1]), -cut_pts], 0)
tf.concat([tf.constant(0.0, shape=[1]), -cut_pts], 0).numpy()

array([ 0.        , -0.38126063], dtype=float32)

* And following results in cumulative sum `b` for 1nd cut_point as `[0,-b1,-b1-b2, -b1-b2-b3. . .-b1-b2-b3-bn]` as mentioned in paper.

In [31]:
b= tf.cumsum(tf.concat([tf.constant(0.0, shape=[1]), -cut_pts], 0))
b.numpy()

array([ 0.        , -0.38126063], dtype=float32)

In [44]:
inputs = tf.constant(x[:5], dtype=tf.float32)
inputs#converting inputs to 'float32' tensors

<tf.Tensor: id=114, shape=(5, 2), dtype=float32, numpy=
array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.4, 0.2]], dtype=float32)>

In [60]:
temperature=0.1
h = tf.matmul(inputs[:,0:1], W) + b
res = tf.nn.softmax(h / temperature)
print('h:', h,'\n\nsoftmax results for feature 1:\n', res)

h: tf.Tensor(
[[1.4       2.4187393]
 [1.4       2.4187393]
 [1.3       2.2187393]
 [1.5       2.6187394]
 [1.4       2.4187393]], shape=(5, 2), dtype=float32) 

softmax results for feature 1:
 tf.Tensor(
[[3.7640464e-05 9.9996233e-01]
 [3.7640464e-05 9.9996233e-01]
 [1.0231069e-04 9.9989772e-01]
 [1.3847483e-05 9.9998617e-01]
 [3.7640464e-05 9.9996233e-01]], shape=(5, 2), dtype=float32)


* **`tf_bin( )` result for `feature 2(Petal width)` i.e., `x[:,1:2]` and `2nd cut_point` or `z[1]`, i.e.,`cut_list[1][1]` from above.**

In [49]:
cut_list

[(0,
  <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.38126063], dtype=float32)>),
 (1,
  <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.08172297], dtype=float32)>)]

In [52]:
cut_pts2= tf.sort(cut_list[1][1])#2nd cutpoint
cut_pts2

<tf.Tensor: id=152, shape=(1,), dtype=float32, numpy=array([0.08172297], dtype=float32)>

* And following results in cumulative sum `b` for 2nd cut_point as `[0,-b1,-b1-b2, -b1-b2-b3. . .-b1-b2-b3-bn]` as mentioned in paper.

In [57]:
b2= tf.cumsum(tf.concat([tf.constant(0.0, shape=[1]), -cut_pts2], 0))
b2.numpy()#b2 is list of cut_points, as in [0,-b1,-b1-b2]

array([ 0.        , -0.08172297], dtype=float32)

In [61]:
temperature=0.1
h2 = tf.matmul(inputs[:,1:2], W) + b2#for feature 2 in input x_ph
res2 = tf.nn.softmax(h2 / temperature)
print('h:', h2,'\n\nsoftmax results for feature 2:\n', res2)

h: tf.Tensor(
[[0.2        0.31827703]
 [0.2        0.31827703]
 [0.2        0.31827703]
 [0.2        0.31827703]
 [0.2        0.31827703]], shape=(5, 2), dtype=float32) 

softmax results for feature 2:
 tf.Tensor(
[[0.23455445 0.76544553]
 [0.23455445 0.76544553]
 [0.23455445 0.76544553]
 [0.23455445 0.76544553]
 [0.23455445 0.76544553]], shape=(5, 2), dtype=float32)


**Now the above softmax values obtained in sequence are passed into following `tf_kron_prod(. . .)` function, both at a once**
* the `reduce(tf_kron_prod, map(lambda z: tf_bin(x[:, z[0]:z[0] + 1], z[1], temperature), enumerate(cut_points_list)))` passes -- computed softmax outputs `res` for `feature 1(Petal length)` i.e., `x[:,0:1]`, cut_point 1 as `cut_list[0][1]`; And `res2` for `feature_2 (Petal width)`, cut_point 2 as `cut_list[1][1]` into **`tf_kron_prod(. . .)`**.
* So the following are the binned values of all 5 input datapoints at the end of `kron_layer` in paper.

In [67]:
from functools import reduce
leaf = reduce(tf_kron_prod, [res, res2])
leaf

<tf.Tensor: id=247, shape=(5, 4), dtype=float32, numpy=
array([[8.8287388e-06, 2.8811724e-05, 2.3454562e-01, 7.6541668e-01],
       [8.8287388e-06, 2.8811724e-05, 2.3454562e-01, 7.6541668e-01],
       [2.3997427e-05, 7.8313256e-05, 2.3453046e-01, 7.6536721e-01],
       [3.2479888e-06, 1.0599494e-05, 2.3455121e-01, 7.6543492e-01],
       [8.8287388e-06, 2.8811724e-05, 2.3454562e-01, 7.6541668e-01]],
      dtype=float32)>

#####  #3.

In [62]:
#3.
def tf_kron_prod(a, b):
    res = tf.einsum('ij,ik->ijk', a, b)
    res = tf.reshape(res, [-1, tf.reduce_prod(res.shape[1:])])
    return res

In [68]:
print('shape of two softamx outputs res & res2:', res.shape, res2.shape)

shape of two softamx outputs res & res2: (5, 2) (5, 2)


* To manually do what `reduce(tf_kron_prod, [res, res2])` is doing: Breaking down `tf_kron_prroduct( )`: 
    * **Inputs:**
    * `res`: softmax output from `tf_bin( )` for `x[:,0:1]` and `cut_pts` value 1 in `cut_points_list`
    * `res2`: softmax output from `tf_bin( )` for `x[:,1:2]` and `cut_pts2` value 2 in `cut_points_list`

In [69]:
kron_res= tf.einsum('ij,ik->ijk', res, res2)
kron_res

<tf.Tensor: id=252, shape=(5, 2, 2), dtype=float32, numpy=
array([[[8.8287388e-06, 2.8811724e-05],
        [2.3454562e-01, 7.6541668e-01]],

       [[8.8287388e-06, 2.8811724e-05],
        [2.3454562e-01, 7.6541668e-01]],

       [[2.3997427e-05, 7.8313256e-05],
        [2.3453046e-01, 7.6536721e-01]],

       [[3.2479888e-06, 1.0599494e-05],
        [2.3455121e-01, 7.6543492e-01]],

       [[8.8287388e-06, 2.8811724e-05],
        [2.3454562e-01, 7.6541668e-01]]], dtype=float32)>

* In above `tf.einsum`, kron product is performed; wherein each softmax value of a feature is multiplied with each other softmax value for other feature.
* Therefore in above--there are two softmax outputs for each feature,
each softmax value for feature 1 is multiplied with each of the softmax values of feature 2. i.e. `kron_res[0][0]` correspondes to product between `res[0][0], res2[0][0]` and `res[0][0], res2[0][1]`; Similarly `kron_res[0][1]` corresponds to product between `res[0][1], res2[0][0]` and `res[0][0], res2[0][1]`.

As in
**kron_res[0][0][0]= res[0][0]* res2[0][0];** likewise
**kron_res[0][0][1]= res[0][0]* res2[0][1]** above.

* kron_product is then reshaped into (None, 4), corresponding to total number of leaves in layer before predicition.

In [73]:
tf.reduce_prod(kron_res.shape[1:]).numpy()

4

In [74]:
kron_res= tf.reshape(kron_res, [-1, tf.reduce_prod(kron_res.shape[1:])])
kron_res

<tf.Tensor: id=283, shape=(5, 4), dtype=float32, numpy=
array([[8.8287388e-06, 2.8811724e-05, 2.3454562e-01, 7.6541668e-01],
       [8.8287388e-06, 2.8811724e-05, 2.3454562e-01, 7.6541668e-01],
       [2.3997427e-05, 7.8313256e-05, 2.3453046e-01, 7.6536721e-01],
       [3.2479888e-06, 1.0599494e-05, 2.3455121e-01, 7.6543492e-01],
       [8.8287388e-06, 2.8811724e-05, 2.3454562e-01, 7.6541668e-01]],
      dtype=float32)>

* following computes the target label as a result of vector product between output at DT leaves and Weight matrix which maps DT leaves & class labels.

In [76]:
#In nn_decision_tree( ) line2
y_pred = tf.matmul(leaf, leaf_score)
y_pred

<tf.Tensor: id=286, shape=(5, 3), dtype=float32, numpy=
array([[0.2296738 , 0.36174935, 0.4942583 ],
       [0.2296738 , 0.36174935, 0.4942583 ],
       [0.22968492, 0.36174738, 0.494249  ],
       [0.22966973, 0.3617501 , 0.49426177],
       [0.2296738 , 0.36174935, 0.4942583 ]], dtype=float32)>

* results from main function

In [78]:
y_pred = nn_decision_tree(inputs, cut_points_list, leaf_score, temperature=0.1)
#loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(logits=y_pred, onehot_labels=y_ph))
y_pred

<tf.Tensor: id=373, shape=(5, 3), dtype=float32, numpy=
array([[0.2296738 , 0.36174935, 0.4942583 ],
       [0.2296738 , 0.36174935, 0.4942583 ],
       [0.22968492, 0.36174738, 0.494249  ],
       [0.22966973, 0.3617501 , 0.49426177],
       [0.2296738 , 0.36174935, 0.4942583 ]], dtype=float32)>

* **Obtaining bins ouputs values for a specific a input feature**

[Run the following]

In [79]:
import tensorflow as tf
from tensorflow.keras import layers
from functools import reduce
tf.__version__

'2.0.0'

In [80]:
from keras.utils import to_categorical
from sklearn.datasets import load_iris

iris= load_iris()

x= iris.data
y=iris.target

x= x[:,2:4]#taking Petal width, Petal width
y = to_categorical(y)
print('X & y shapes:', x.shape, y.shape)

Using TensorFlow backend.


X & y shapes: (150, 2) (150, 3)


In [81]:
def binn(x, cut_points, temperature):        
    # x is a N-by-1 matrix (column vector)
    # cut_points is a D-dim vector (D is the number of cut-points)
    # this function produces a N-by-(D+1) matrix, each row has only one element being one and the rest are all zeros
    D = cut_points.get_shape().as_list()[0]
    W = tf.reshape(tf.linspace(1.0, D + 1.0, D + 1), [1, -1])#corresponds to list of no. of cut_points
    #Or use tf.Variable(tf.reshape(tf.linspace(1.0, D + 1.0, D + 1), [1, -1]), trainable=False)
        
    cut_points = tf.sort(cut_points)  # makes sure cut_points is monotonically increasing
    b = tf.cumsum(tf.concat([tf.constant(0.0, shape=[1]), -cut_points], 0))#outputs list os cutpoints as [0,-b1,-b1]
        
    h = tf.matmul(x, W) + b
    res = tf.nn.softmax(h / temperature)
    return res

#x[:2]
temperature=0.1
cut_points_list = [tf.Variable(tf.random.uniform([i])) for i in [1,1]]
c_pt= list(enumerate(cut_points_list))
c_pt

[(0,
  <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.5497626], dtype=float32)>),
 (1,
  <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.24273801], dtype=float32)>)]

In [82]:
c_pt[0][1]

<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.5497626], dtype=float32)>

In [83]:
cut_points = tf.sort(c_pt[0][1])  # makes sure cut_points is monotonically increasing
b = tf.cumsum(tf.concat([tf.constant(0.0, shape=[1]), -cut_points], 0))
b

<tf.Tensor: id=422, shape=(2,), dtype=float32, numpy=array([ 0.       , -0.5497626], dtype=float32)>

In [84]:
D= c_pt[0][1].get_shape().as_list()[0]#D= 1 here
W = tf.reshape(tf.linspace(1.0, D + 1.0, D + 1), [1, -1])
W

<tf.Tensor: id=428, shape=(1, 2), dtype=float32, numpy=array([[1., 2.]], dtype=float32)>

In [85]:
#import numpy as np
#np.dtype(x[0][0])#x[:2,0:1]) ##To check the dtype of input
inputs = tf.constant(x[:5], dtype=tf.float32)
inputs

<tf.Tensor: id=429, shape=(5, 2), dtype=float32, numpy=
array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.4, 0.2]], dtype=float32)>

In [86]:
h = tf.matmul(inputs[:,0:1], W) + b
h

<tf.Tensor: id=435, shape=(5, 2), dtype=float32, numpy=
array([[1.4      , 2.2502375],
       [1.4      , 2.2502375],
       [1.3      , 2.0502372],
       [1.5      , 2.4502373],
       [1.4      , 2.2502375]], dtype=float32)>

* Softmax outputs taking into account cut_points per feature and input feature column.

* a1 : corresponds to softmax outputs for input feature 1 & cut_point 1

In [88]:
a1= binn(inputs[:,0:1], c_pt[0][1], temperature)
a1#softmax outputs (batch_size, num_bins)

<tf.Tensor: id=505, shape=(5, 2), dtype=float32, numpy=
array([[2.0294457e-04, 9.9979705e-01],
       [2.0294457e-04, 9.9979705e-01],
       [5.5146980e-04, 9.9944848e-01],
       [7.4668860e-05, 9.9992537e-01],
       [2.0294457e-04, 9.9979705e-01]], dtype=float32)>

* a2 : corresponds to softmax outputs for input feature 2 & cut_point 2

In [90]:
a2= binn(inputs[:, 1:2], c_pt[1][1], temperature)
a2#softmax outputs (batch_size, num_bins)

<tf.Tensor: id=544, shape=(5, 2), dtype=float32, numpy=
array([[0.6052479 , 0.39475214],
       [0.6052479 , 0.39475214],
       [0.6052479 , 0.39475214],
       [0.6052479 , 0.39475214],
       [0.6052479 , 0.39475214]], dtype=float32)>

* Obtaining kron outputs for specific inputs

In [91]:
def kron_prod(a, b):
    res = tf.einsum('ij,ik->ijk', a, b)
    res = tf.reshape(res, [-1, tf.math.reduce_prod(res.shape[1:])])
    return res

a11= a1[:1].numpy()
a21= a2[:1].numpy()
print('softmax values for 1st input at the end of binning layer:\n', a11, a21)


softmax values for 1st input at the end of binning layer:
 [[2.0294457e-04 9.9979705e-01]] [[0.6052479  0.39475214]]


* Computes, aligns the dot product of softmax val. of each feature with every other feature's softmax vals. along rows 

In [93]:
tf.einsum('ij,ik->ijk', a1[:1],a2[:1])

<tf.Tensor: id=578, shape=(1, 2, 2), dtype=float32, numpy=
array([[[1.2283179e-04, 8.0112804e-05],
        [6.0512507e-01, 3.9467204e-01]]], dtype=float32)>

In [94]:
kron_prod(a1[:1],a2[:1]).numpy()# Same as reduce(kron_prod, [a1[:1],a2[:1]])

array([[1.2283179e-04, 8.0112804e-05, 6.0512507e-01, 3.9467204e-01]],
      dtype=float32)

* Kron product : manual product each softmax output of a feature with  each other output of other feature

In [95]:
print('1st element in kron_prod:',a11[0][0]*a21[0][0],'\nlast element in kron_prod:',a11[0][1]*a21[0][1])
print('Which is same as tf.einsum')

1st element in kron_prod: 0.00012283179 
last element in kron_prod: 0.39467204
Which is same as tf.einsum


* If there were 3 input features & the cuts_per_feature were be [1,1,1]

In [97]:
reduce(kron_prod, [a1[:1],a2[:1],a1[:1]])#Same as kron_prod(kron_prod(a1,a2), a1)
##that is first elemenet is computed as:
#kron_prod(a1[:1],a2[:1]).numpy()[0][0] * a11[0][0]

<tf.Tensor: id=685, shape=(1, 8), dtype=float32, numpy=
array([[2.4928044e-08, 1.2280686e-04, 1.6258459e-08, 8.0096543e-05,
        1.2280684e-04, 6.0500228e-01, 8.0096550e-05, 3.9459193e-01]],
      dtype=float32)>